In [ ]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
from google.colab import files
files.upload()

In [ ]:
pip install python-dotenv

In [ ]:
# Initial imports
import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer



%matplotlib inline

In [ ]:
df = pd.read_csv('223k crypto news headlines. Dataset. BDCenter Digital.csv')
df

In [ ]:
def getPolarity(newstext):
  return TextBlob(newstext).sentiment.polarity

df['Polarity'] = df['headlinetext'].apply(getPolarity)
df

In [ ]:
df.isnull().any()

In [ ]:
pip install --user scipy wordcloud nltk seaborn textblob

In [ ]:
import numpy as np
import pandas as pd
import json, nltk
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
# nltk.download('wordnet')   # for Lemmatization

%matplotlib inline

In [ ]:
df

In [ ]:
#df['Polarity'] = pd.to_numeric(df['Polarity'],downcast='integer')
df['Polarity'] = df['Polarity'].astype(int)

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
import re

def process_text(text):
    text = text.lower()                                             # Lowercases the string
    text = re.sub('@[^\s]+', '', text)                              # Removes usernames
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', text)   # Remove URLs
    text = re.sub(r"\d+", " ", str(text))                           # Removes all digits
    text = re.sub('&quot;'," ", text)                               # Remove (&quot;) 
                                               # Replaces Emojis
    text = re.sub(r"\b[a-zA-Z]\b", "", str(text))                   # Removes all single characters
    for word in text.split():
        #if word.lower() in contractions:
            #text = text.replace(word, contractions[word.lower()])   # Replaces contractions
            text = re.sub(r"[^\w\s]", " ", str(text))                       # Removes all punctuations
            text = re.sub(r'(.)\1+', r'\1\1', text)                         # Convert more than 2 letter repetitions to 2 letter
            text = re.sub(r"\s+", " ", str(text))                           # Replaces double spaces with single space    
    return text

In [ ]:
#df['processed_text'] = np.vectorize(process_text)(df[text])
df['processed_text'] = np.vectorize(process_text)(df['headlinetext'])

In [ ]:
df.head(10)

In [ ]:
#Tokenization
tokenized_text = df['processed_text'].apply(lambda x: x.split())
tokenized_text.head()

In [ ]:
#STEMMING

#from nltk.stem.porter import *
#stemmer = PorterStemmer()

#tokenized_text = tokenized_text.apply(lambda x: [stemmer.stem(i) for i in x])
#tokenized_text.head()

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

tokenized_text = tokenized_text.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])
tokenized_text.head()

In [ ]:
#nltk.download("stopwords")
#from nltk.corpus import stopwords
#stop_words = set(stopwords.words('english'))
#stop_words, nltk.corpus.stopwords.words('english')

In [ ]:
#for i in range(len(tokenized_text)):
    
     # Below code is used for no stop word removal
     #tokenized_text[i] = ' '.join(tokenized_text[i]) 
    
     # Used for stop word removal 
     # (Below is commented out as sentiment analysis is giving better accuracy without removing stop words.
     # If you still want to check, comment out the above line, uncomment the line below and run the code again.)
    
     #tokenized_text[i] = ' '.join([word for word in tokenized_text[i] if word not in stop_words])  
    

#df['processed_text'] = tokenized_text
#df.head()

In [ ]:
#Feature extraction (vectorization)
#1. Count Vectorizer
#2. Tf-Idf vectorizer


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(ngram_range=(1,2))    # Unigram and Bigram
final_vectorized_data = count_vectorizer.fit_transform(df['processed_text'])  
final_vectorized_data

In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer 

#tf_idf_vectorizer = TfidfVectorizer(use_idf=True,ngram_range=(1,3))
#final_vectorized_data = tf_idf_vectorizer.fit_transform(df['processed_text'])

#final_vectorized_data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(final_vectorized_data, df['Polarity'],
                                                    test_size=0.2, random_state=69)

In [ ]:
print("X_train_shape : ",X_train.shape)
print("X_test_shape : ",X_test.shape)
print("y_train_shape : ",y_train.shape)
print("y_test_shape : ",y_test.shape)

In [ ]:
#standardization

from sklearn.preprocessing import StandardScaler

sc = StandardScaler(with_mean=False)

X_train = sc.fit_transform(X_train)

X_test = sc.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Training Accuracy :", model.score(X_train, y_train))

Training Accuracy : 1.0


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))